## CCNSS 2018 Module 5: Whole-Brain Dynamics and Cognition
# Bonus Tutorial: Complex Network Analysis of Research Network


*Please execute the cell bellow in order to initialize the notebook environment*

In [0]:
!if [ ! -d data ]; then git clone https://github.com/ccnss/ccnss2018_students; \
                        cp -rf ccnss2018_students/module5/bonus_research_network/data ./; \
                        cp ccnss2018_students/module5/net_tool.py ./; fi

In [0]:
import matplotlib.pyplot as plt    # import matplotlib
import numpy as np                 # import numpy
import random                      # import basic random number generator functions
import csv                         # import CSV(Comma Separated Values) file reading and writing
import scipy as sp                 # import scipy
from scipy import signal           # import signal module from scipy
from scipy import sparse           # import sparse module from scipy
import os                          # import basic os functions
import time                        # import time to measure real time
import collections                 # import collections
import networkx as nx              # import networkx 

import sys
sys.path.append('../')
import net_tool as net             # import net_tool, a network analysis toolbox from tutorial #1 

data_folder = 'data'

print('Available data files:', ' '.join(os.listdir(data_folder)))

data_file_0 = os.path.join(data_folder, 'research_net_label.txt')
data_file_1 = os.path.join(data_folder, 'research_net_sym_zsc.txt')
data_file_2 = os.path.join(data_folder, 'research_net_p25.txt')
data_file_2 = os.path.join(data_folder, 'research_net_p25.txt')
data_file_2 = os.path.join(data_folder, 'research_net_p25.txt')

# Objectives

In this notebook, we will construct a research network of cnnss2018 participants, using the tools of complex network theory of which we have acquired.




## Background

A network is set of elements with connections between them. We can have insights on many complex systems, by looking at their connection structure.

When making a network from a given data, one has two things to consider:

* What are the nodes?
* How do we link the nodes?

The questions seem simple, but sometimes delicate decision has to be made in the process. For example, when turning weights between links into binary (1 or 0) connections, one has to decide the threshold value.


** EXERCISE 1: Turning weighted network in to binary network **

We have analyzed  the research questionnaires filled by students and tutors and z-scored the values per each person. Then we averaged scores between two people. In essence, we turned directional connection into undirectional connection. 

The z-score matrix between participants are given.

To make a binary network out of given z-score matrix, we need to set a threshold value. If a given link has a value above the threshold, the link will survive and has value of 1 in the connectivity matrix. If not, the value will be 0, which means that there is no link between them.

• One must determine a right threshold value for the matrix. For example, you can choose a value such that highest 30% of the z-scored values between nodes will turn into connection. In such case, the *threshold* is 0.3 (or 30%).

• In this exercise, construct a function that thresholds given connectivity matrix, turnning it into a binary matrix. 

• Call the constructed function, and make a binary connectivity matrix with threshold = 0.3 (30%).


In [0]:
# importing the symmetrized z-score matrix

net_school_sym_zsc = [ row for row in csv.reader(open(data_file_1,'r'),delimiter='\t')]
net_school_sym_zsc = np.array(net_school_sym_zsc).astype(float)

In [0]:
# making the label of the network

#labels =[ row for row in csv.reader(open(data_file_0,'r')) ]

labels={} ;
labels[0]=r'$Xinke$' ; labels[1]=r'$Daming$' ; labels[2]=r'$Marco W$' ; labels[3]=r'$Guang-Yao$' ; labels[4]=r'$Sofia$' ; labels[5]=r'$Homero$';
labels[6]=r'$Lin$' ; labels[7]=r'$James$' ; labels[8]=r'$Asma$' ; labels[9]=r'$Xiaohan$' ; labels[10]=r'$Dario$' ; labels[11]=r'$Maaike$';
labels[12]=r'$Zhongqi$' ; labels[13]=r'$Beata$' ; labels[14]=r'$Vincent$' ; labels[15]=r'$Amna$' ; labels[16]=r'$Feng$' ; labels[17]=r'$Maryam$';
labels[18]=r'$Matthias$' ; labels[19]=r'$Jiang$' ; labels[20]=r'$Giulio$' ; labels[21]=r'$Aurelio$' ; labels[22]=r'$Jorge$' ; labels[23]=r'$Mingyu$';
labels[24]=r'$Alice$' ; labels[25]=r'$Yang$' ; labels[26]=r'$Mathew$' ; labels[27]=r'$Manel$' ; labels[28]=r'$Joon$' ; labels[29]=r'$Yue$';
labels[30]=r'$Marco B$' ; labels[31]=r'$Karla$' ; labels[32]=r'$Luqun$' ; labels[33]=r'$Junru$' ; labels[34]=r'$Ainhoa$' ; labels[35]=r'$Zhewei$';
labels[36]=r'$Alex$' ; labels[37]=r'$Ziqing$'

mapping={0:'Xinke',1:'Daming',2:'Marco W',3:'Guang-Yao',4:'Sofia',5:'Homero',6:'Lin',7:'James',8:'Asma',9:'Xiaohan',10:'Dario',11:'Maaike',
         12:'Zhongqi',13:'Beata',14:'Vincent',15:'Amna',16:'Feng',17:'Maryam',18:'Matthias',19:'Jiang',20:'Giulio',21:'Aurelio',22:'Jorge',
         23:'Mingyu',24:'Alice',25:'Yang',26:'Mathew',27:'Manel',28:'Joon',29:'Yue',30:'Marco B',31:'Karla',32:'Luqun',33:'Junru',34:'Ainhoa',35:'Zhewei',
         36:'Alex',37:'Ziqing' }


In [0]:
# code for generating binary connectivity matrix, from a weighted connectivity matrix

def cal_mat_thresholded(data_mat, threshold):

    prob = 1 - threshold
    data_mat_nz = data_mat*(data_mat>0)
    data_value_sorted = np.sort(data_mat_nz,axis=None)
    #print(data_value_sorted)
    data_value_threshold = data_value_sorted[int(np.round(len(data_value_sorted)*prob))]
    #print(data_value_threshold)
    data_mat_thresholded = data_mat*(data_mat>=data_value_threshold)
    
    data_mat_binary = np.ceil(data_mat_thresholded/(np.max(data_mat_thresholded))).astype(int)
    return data_mat_binary

In [0]:
# call above function and generate the binary connectivity matrix

threshold = 0.3
net_school_mat = cal_mat_thresholded(net_school_sym_zsc,threshold)
#print(np.sum(net_school_mat))

net_school_mat_N = len(net_school_mat)
net_school_mat_l = np.sum(net_school_mat)/2

print('# of nodes in research network N:', net_school_mat_N)
print('# of links in research network l:', net_school_mat_l)

**EXPECTED OUTPUT**
```
# of nodes in research network N: 38
# of links in research network l: 217.0
```

** EXERCISE 2: Applying network measures**

Now, we can apply various network measures that we learned in previous tutorial to our network.

• using, `net_degree_plot`, plot the degree distribution.

• Apply network measures to this network, such as $C$ and $b$.

• You can draw the network, using `nx.from_numpy_matrix` and `nx.draw`. 

In [0]:
net.net_degree_plot(net_school_mat,'school network')

In [0]:
net_school_ci = np.zeros([net_school_mat_N])
for i in range(net_school_mat_N):
    net_school_ci[i] = net.clustering_coef(net_school_mat,i)
net_school_c_mean = np.mean(net_school_ci)
print('C of research network:',net_school_c_mean)

net_school_b = net.betweenness_central(net_school_mat,normalized=False)
net_school_b_mean = np.mean(net_school_b)
print('b of research network:',net_school_b_mean)

In [0]:
net_school_mat_d = net.net_mat_distance(net_school_mat)
L, E = net.net_L_E(net_school_mat_d)

print('L:', L,'\t', 'E:', E)

In [0]:


Graph_net_school = nx.from_numpy_matrix(net_school_mat)
H=nx.relabel_nodes(Graph_net_school,mapping)
pos = nx.spring_layout(H)


plt.figure(3,figsize=(15,10)) 
nx.draw(H,pos,with_labels=True,node_size=2000,node_color='white',font_size=18,font_color="black",font_weight='bold',node_shape='o',alpha=0.95)
plt.show()
plt.savefig('school_net.png')

**EXPECTED OUTPUT**

```
C of research network: 0.39031564139515906
b of research network: 13.552631578947368
```

** EXERCISE 3: Changing the threshold**

Now, we can change the threshold value and observe the changes in the network.

• For example, one can change the threshold to 0.25 or 0.35, then apply network measures to this network, such as $C$ and $b$.

• You can draw the networks with different thresholds, and observe how they differ.